In [1]:
import ee
import os
import geemap
import json
from shapely.geometry import shape
import geopandas as gpd

In [2]:
ee.Authenticate()
ee.Initialize()

In [3]:
project_dir = 'projects/alpod-412314/assets/' 

region = 'AKCP'
region_path = f'{region}_weekly'
folder = project_dir + region_path

#print(ee.data.listAssets(project_dir))

In [4]:
weekly_mosaics = ee.data.listAssets(project_dir + 'region_weekly')

In [5]:
img = ee.Image('projects/alpod-412314/assets/region_weekly/AKCP_2016_29')
lakes_path = f'{project_dir}Lake_extractions/{region}_extraction'
lake_polygons = ee.FeatureCollection(lakes_path)
#print(lake_polygons.first().getInfo())

In [6]:
obs_mask = img.unmask(0)
obs_mask = obs_mask.rename('observed')
#print(obs_mask.getInfo())

img_wtr = ee.Image.constant(0)
img_wtr = img_wtr.where(img.select('water_occurance').eq(1), 2)

img_land_ice_cloud = ee.Image.constant(0)
img_land_ice_cloud = img_land_ice_cloud.where(img.select('water_occurance').eq(0), 1)

In [21]:
obs_mask = img.unmask(0)
obs_mask = obs_mask.rename('observed')
#print(obs_mask.getInfo())

lakes_binary = lake_polygons.reduceToImage(
    properties=['n_lakes'],
    reducer=ee.Reducer.anyNonZero()
).neq(0)
lakes_binary = lakes_binary.rename('buffered_lake')
#print(lakes_binary.getInfo())

# 100: Observed water
# 50: Observed land
# -50: Cloudy or ice
# -100: Unobserved

expr = """
(water_occurance == 1) ? 100 :
((water_occurance != 1) && (buffered_lake == 1)) ? 50 :
-100
"""

conditional = lakes_binary.expression(
    expr,
    {
        'observed_data': obs_mask.select('observed'),
        'buffered_lake': lakes_binary.select('buffered_lake'),
        'water_occurance': img_wtr.select('water_occurance'),
        'ice_land_cloud': img_land_ice_cloud.select('water_occurance')
    }
)
print(conditional.getInfo())



{'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -100, 'max': 100}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [9]:
obs_viz = {
    'bands': ['observed'],
    'min': 0,
    'max': 1,
    'palette': ['red', 'green']
}

lakes_viz = {
    'bands': ['buffered_lake'],
    'min': 0,
    'max': 1, 
    'palette': ['white', 'blue']
}

value_to_color = {
    100: 'blue',   # Observed water
    50: 'brown',    # Observed land
    -50: 'pink',    # Cloudy or ice
    -100: 'white'  # Unobserved
}

conditional_palette = [value_to_color[v] for v in sorted(value_to_color.keys())]


conditional_viz = {
    'min': min(value_to_color.keys()),
    'max': max(value_to_color.keys()), 
    'palette': conditional_palette,
}

# Create a map to display the results
Map2 = geemap.Map(center=(65, -135), zoom=4)
Map2.addLayer(obs_mask, obs_viz, 'Observation Mask')
#Map2.addLayer(conditional, conditional_viz, 'Conditional Test')
# Map2.addLayer(lakes_binary, lakes_viz, 'Buffered Lakes')

qk_viz = {
    'bands': ['constant'],
    'min': 0,
    'max': 2, 
    'palette': ['white', 'blue']
}
qk_viz2 = {
    'bands': ['constant'],
    'min': 0,
    'max': 1, 
    'palette': ['white', 'brown']
}


Map2.addLayer(img_wtr, qk_viz, 'huhhh')
Map2.addLayer(img_land_ice_cloud, qk_viz2, 'land ho?')

Map2

Map(center=[65, -135], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [28]:
print(conditional_viz)

{'min': -100, 'max': 100, 'palette': ['white', 'pink', 'brown', 'blue']}
